In [1]:
# Cell 1: 초기 설정, 임포트 및 경로 정의
# -*- coding: utf-8 -*-
# Jupyter Notebook: full_pipeline.ipynb - Cell 1

import os
import sys
import logging
import traceback
import time
import itertools
import json
import numpy as np
import scipy.sparse as sp
import re # JSON 파싱 시 정규식 사용 위해 추가

# --- 프로젝트 루트 설정 (⭐️⭐️⭐️ 노트북 파일 위치에 맞게 수정 필수 ⭐️⭐️⭐️) ---
# 예: 노트북이 kospi_relation_rag_pipeline 폴더 내에 저장되어 있다면 아래 코드 사용 가능
# project_root = os.path.abspath(os.path.join(os.getcwd()))
# 예: 노트북이 프로젝트 폴더 밖에 있다면 절대 경로 지정
project_root = os.path.abspath(os.path.join(os.getcwd())) # !!! 사용자 환경에 맞게 수정 !!!
if project_root not in sys.path:
    sys.path.append(project_root)
# --------------------------------------------------------------------------

# --- 필요한 모듈 및 설정 임포트 ---
try:
    from core import document_processor, vector_store_manager, llm_extractor, matrix_builder
    from core.index_setup import create_search_index # 인덱스 생성 함수
    from config import ( # 필요한 모든 설정값 임포트
        logger, KOSPI100_LIST_PATH, TARGET_YEAR, DATA_DIR,
        AZURE_DI_ENDPOINT, AZURE_DI_KEY, AZURE_DI_MODEL_ID,
        AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY, AZURE_OPENAI_API_VERSION,
        AZURE_OPENAI_EMBEDDING_DEPLOYMENT, AZURE_OPENAI_CHAT_DEPLOYMENT, # Chat 배포명 필요
        AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_KEY, AZURE_SEARCH_INDEX_NAME,
        CHUNK_SIZE, CHUNK_OVERLAP, TOP_K_RESULTS,
        STRENGTH_MAPPING, RETRY_ATTEMPTS, RETRY_WAIT_SECONDS
    )
    # OpenAI, Search 클라이언트 초기화 함수 임포트
    from core.vector_store_manager import initialize_openai_client, initialize_search_client
except ImportError as e:
    print(f"모듈 임포트 오류: {e}")
    print(f"프로젝트 경로 설정(project_root='{project_root}') 또는 가상환경, 파일 존재 여부를 확인하세요.")
    raise

# --- 경로 정의 ---
INPUT_PDF_DIR = os.path.join(DATA_DIR, 'input_pdfs')
OUTPUT_MATRIX_DIR = os.path.join(project_root, "output_matrices")
os.makedirs(INPUT_PDF_DIR, exist_ok=True) # 입력 폴더도 생성 확인
os.makedirs(OUTPUT_MATRIX_DIR, exist_ok=True)

# --- 실행 규모 설정 ---
IS_TEST_RUN = True  # <--- 테스트 시 True, 전체 실행 시 False로 변경
NUM_COMPANIES_LIMIT = 5 if IS_TEST_RUN else 100 # 테스트 시 처리할 최대 기업 수 (최대 100)
# --------------------

# --- 전역 변수 또는 설정 ---
API_CALL_DELAY_SECONDS = 0.5 # API 호출 사이 지연 시간 (초) - Rate Limit 방지용 (조절 필요)

logger.info("===== 노트북 셀 1: 초기 설정 및 임포트 완료 =====")
print(f"프로젝트 루트: {project_root}")
print(f"PDF 입력 디렉토리: {INPUT_PDF_DIR}")
print(f"매트릭스 출력 디렉토리: {OUTPUT_MATRIX_DIR}")
print(f"실행 모드: {'TEST' if IS_TEST_RUN else 'FULL'} ({NUM_COMPANIES_LIMIT}개 기업 처리)")
print(f"사용할 임베딩 배포명: {AZURE_OPENAI_EMBEDDING_DEPLOYMENT}")
print(f"사용할 채팅 배포명: {AZURE_OPENAI_CHAT_DEPLOYMENT}")
print(f"사용할 검색 인덱스: {AZURE_SEARCH_INDEX_NAME}")

2025-04-18 14:08:49,111 - config - INFO - ===== 노트북 셀 1: 초기 설정 및 임포트 완료 =====


프로젝트 루트: /Users/nsj/Desktop/kospi_relation_rag_pipeline
PDF 입력 디렉토리: /Users/nsj/Desktop/kospi_relation_rag_pipeline/data/input_pdfs
매트릭스 출력 디렉토리: /Users/nsj/Desktop/kospi_relation_rag_pipeline/output_matrices
실행 모드: TEST (5개 기업 처리)
사용할 임베딩 배포명: text-embedding-ada-002
사용할 채팅 배포명: gpt-35-turbo
사용할 검색 인덱스: kospi100-rag-index


In [2]:
# Cell 2: Azure 클라이언트 초기화
# Jupyter Notebook: full_pipeline.ipynb - Cell 2

logger.info("--- Azure 클라이언트 초기화 시작 ---")

# OpenAI 클라이언트 (Embedding & Chat 공용)
# vector_store_manager 내 initialize_openai_client 사용 (AzureOpenAI 클라이언트 반환 가정)
openai_client = initialize_openai_client()

# AI Search 클라이언트
# vector_store_manager 내 initialize_search_client 사용
search_client = initialize_search_client()

# Document Intelligence 클라이언트는 필요 시 document_processor 내부에서 초기화됨
# 만약 미리 초기화해서 재사용하려면 여기서 초기화
# di_client = document_processor.initialize_di_client()


if not all([openai_client, search_client]): # 필요시 di_client 추가
    logger.error("하나 이상의 Azure 클라이언트 초기화에 실패했습니다. .env 파일 및 Azure 서비스 상태를 확인하세요.")
    raise ConnectionError("Azure Client 초기화 실패")
else:
    logger.info("--- OpenAI, Search 클라이언트 초기화 완료 ---")

print("OpenAI Client Initialized:", openai_client is not None)
print("Search Client Initialized:", search_client is not None)

2025-04-18 14:08:51,704 - config - INFO - --- Azure 클라이언트 초기화 시작 ---
2025-04-18 14:08:51,765 - config - INFO - Azure OpenAI 클라이언트 초기화 성공
2025-04-18 14:08:51,767 - config - INFO - Azure AI Search 클라이언트 초기화 성공 (인덱스: kospi100-rag-index)
2025-04-18 14:08:51,767 - config - INFO - --- OpenAI, Search 클라이언트 초기화 완료 ---


OpenAI Client Initialized: True
Search Client Initialized: True


In [3]:
# Cell 3: Azure AI Search 인덱스 생성/확인
# Jupyter Notebook: full_pipeline.ipynb - Cell 3

# 이 셀은 처음 실행하거나 인덱스 스키마 변경 시 필요. 이후에는 주석처리 가능.
logger.info("--- Azure AI Search 인덱스 설정 확인/시도 ---")
index_ready = create_search_index() # core.index_setup 모듈의 함수 호출

if index_ready:
    logger.info("--- 인덱스 준비 완료 ---")
else:
    logger.error("--- 인덱스 설정 실패. 이후 단계 진행 불가 ---")
    raise RuntimeError("Azure AI Search 인덱스 설정 실패")

print(f"인덱스 '{AZURE_SEARCH_INDEX_NAME}' 준비 상태: {index_ready}")

2025-04-18 14:08:54,091 - config - INFO - --- Azure AI Search 인덱스 설정 확인/시도 ---
2025-04-18 14:08:54,094 - config - INFO - Azure AI Search 인덱스 'kospi100-rag-index' 생성 또는 업데이트 시도...
2025-04-18 14:08:54,095 - azure.search.documents.indexes._generated._serialization - WARNING - retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
2025-04-18 14:08:54,097 - azure.search.documents.indexes._generated._serialization - WARNING - retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
2025-04-18 14:08:54,098 - azure.search.documents.indexes._generated._serialization - WARNING - retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored
2025-04-18 14:08:54,098 - azure.search.documents.indexes._generated._serialization - WARNING - retrievable is not a known attribute of cl

인덱스 'kospi100-rag-index' 준비 상태: True


In [10]:
# Jupyter Notebook: analysis_notebook.ipynb - Cell 4 (수정됨)
import pandas as pd # pandas 임포트 추가

# KOSPI 100 종목 코드 목록 로드 (Cell 1에서 project_root 정의됨 가정)
stock_codes_from_file = []
if not os.path.exists(KOSPI100_LIST_PATH):
    logger.error(f"KOSPI 100 목록 파일을 찾을 수 없습니다: {KOSPI100_LIST_PATH}")
else:
    try:
        with open(KOSPI100_LIST_PATH, 'r', encoding='utf-8') as f:
            all_codes_from_file = [line.strip() for line in f if line.strip() and not line.startswith('#')]
        logger.info(f"파일에서 종목 코드 {len(all_codes_from_file)}개 로드됨.")
        # 설정된 개수만큼만 최종 처리 대상 리스트 생성 (Cell 1의 NUM_COMPANIES_LIMIT 사용)
        stock_codes = all_codes_from_file[:NUM_COMPANIES_LIMIT]
        logger.info(f"처리 대상 종목 코드 {len(stock_codes)}개 선택됨 (최대 {NUM_COMPANIES_LIMIT}개).")
    except Exception as e:
        logger.error(f"종목 코드 파일 로드 중 오류 발생: {e}")
        stock_codes = [] # 오류 시 빈 리스트

if not stock_codes:
     logger.error("처리할 종목 코드가 없습니다. KOSPI100_LIST_PATH 파일을 확인하세요.")
     raise ValueError("처리 대상 종목 코드가 없습니다.")

# --- ⭐️ 회사 이름 매핑 로드 (추가된 부분) ⭐️ ---
mapping_file_path = os.path.join(DATA_DIR, 'kospi100_name_map.xlsx') # 매핑 파일 경로
code_to_name = {} # 초기화 (stock_code -> company_name_ko)
name_to_code = {} # 초기화 (company_name_ko -> stock_code)

if os.path.exists(mapping_file_path):
    try:
        # CSV 파일을 읽어 DataFrame 생성 (stock_code는 문자열로 읽음)
        mapping_df = pd.read_excel(mapping_file_path, dtype={'stock_code': str})
        # 필요한 컬럼 존재 확인
        if 'stock_code' in mapping_df.columns and 'company_name_ko' in mapping_df.columns:
            # stock_code를 키로, company_name_ko를 값으로 하는 딕셔너리 생성
            code_to_name = mapping_df.set_index('stock_code')['company_name_ko'].to_dict()
            # company_name_ko를 키로, stock_code를 값으로 하는 딕셔너리 생성
            name_to_code = mapping_df.set_index('company_name_ko')['stock_code'].to_dict()
            logger.info(f"회사 이름 매핑 로드 성공: {len(code_to_name)}개 기업")
            # print("로드된 매핑 샘플:", list(code_to_name.items())[:5])
        else:
             logger.error(f"매핑 파일({mapping_file_path})에 'stock_code' 또는 'company_name_ko' 컬럼이 없습니다.")
             logger.warning("종목 코드를 회사 이름 대신 사용합니다.")
    except Exception as e:
        logger.error(f"회사 이름 매핑 파일 로드/처리 오류: {e}")
        logger.warning("종목 코드를 회사 이름 대신 사용합니다.")
else:
    logger.warning(f"회사 이름 매핑 파일({mapping_file_path})을 찾을 수 없습니다. 종목 코드를 이름 대신 사용합니다.")
# --- 매핑 로드 끝 ---

# Matrix 생성을 위한 종목 코드 -> 인덱스 매핑 생성 (현재 처리 대상 기준)
company_to_idx = {code: i for i, code in enumerate(stock_codes)}
idx_to_company = {i: code for i, code in enumerate(stock_codes)}
num_companies = len(stock_codes) # 처리 대상 기업 수

print(f"처리 대상 종목 코드 (총 {num_companies}개): {stock_codes}")
print(f"로드된 회사 이름 매핑 수: {len(code_to_name)}")
# print(f"회사->인덱스 매핑 (예시): {list(company_to_idx.items())[:5]}")

2025-04-18 14:24:49,321 - config - INFO - 파일에서 종목 코드 100개 로드됨.
2025-04-18 14:24:49,322 - config - INFO - 처리 대상 종목 코드 5개 선택됨 (최대 5개).
2025-04-18 14:24:52,460 - config - INFO - 회사 이름 매핑 로드 성공: 100개 기업


처리 대상 종목 코드 (총 5개): ['005930', '000660', '105560', '005380', '012450']
로드된 회사 이름 매핑 수: 100


In [6]:
#Cell 5: [PART 1] 데이터 인덱싱 루프
# Jupyter Notebook: full_pipeline.ipynb - Cell 5

logger.info(f"===== PART 1: 데이터 인덱싱 시작 ({num_companies}개 기업) =====")
failed_ingestion = [] # 인덱싱 실패한 기업 목록
successful_ingestion_count = 0

for idx, stock_code in enumerate(stock_codes):
    current_stock_code = stock_code
    logger.info(f"--- 인덱싱 시작 ({idx+1}/{num_companies}): {current_stock_code} ---")
    # 변수 초기화
    pdf_content_bytes = None
    pdf_file_path = None
    di_result = None
    sections = {}
    full_text = ""
    text_chunks = []
    valid_chunks = []
    embeddings = []
    upload_success = False
    status = "성공" # 기본 상태

    try:
        # 1. PDF 파일 읽기
        pdf_file_name = f"{current_stock_code}.pdf" # 파일 이름 형식 확인!
        pdf_file_path = os.path.join(INPUT_PDF_DIR, pdf_file_name)
        if not os.path.exists(pdf_file_path):
            logger.warning(f"PDF 파일을 찾을 수 없음: {pdf_file_path}. 건너뜁니다.")
            status = "PDF 없음"
            failed_ingestion.append({"code": current_stock_code, "reason": "PDF Not Found"})
            time.sleep(0.1)
            continue # 다음 기업으로

        with open(pdf_file_path, "rb") as f:
            pdf_content_bytes = f.read()
        logger.info(f"PDF 로드 성공: {pdf_file_path} ({len(pdf_content_bytes)} bytes)")

        # 2. Document Intelligence 분석
        if pdf_content_bytes:
            # analyze_document_content 함수가 바이트를 받는지 확인
            di_result = document_processor.analyze_document_content(pdf_content_bytes)
            time.sleep(API_CALL_DELAY_SECONDS) # DI 호출 후 딜레이

        if not di_result or not di_result.content:
            logger.warning(f"DI 분석 실패 또는 내용 없음. 건너뜁니다.")
            status = "DI 실패"
            failed_ingestion.append({"code": current_stock_code, "reason": "DI Analysis Failed"})
            continue

        # 3. 섹션 추출 및 전체 텍스트 생성
        sections = document_processor.extract_sections_from_di_result(di_result) # 필터링 없는 버전
        if sections:
            full_text = "\n\n".join(sections.values())
        else:
            logger.warning("섹션(H2/H3) 추출 실패. 전체 Markdown 내용을 사용합니다.")
            full_text = di_result.content

        if not full_text or not full_text.strip():
            logger.warning(f"유효한 텍스트 내용 없음. 건너뜁니다.")
            status = "텍스트 없음"
            failed_ingestion.append({"code": current_stock_code, "reason": "No Text Content"})
            continue
        logger.info(f"텍스트 추출 완료 (길이: {len(full_text)} 자)")

        # 4. 텍스트 청킹
        text_chunks = vector_store_manager.get_text_chunks(full_text)
        if not text_chunks:
            logger.warning(f"텍스트 청킹 결과 없음. 건너뜁니다.")
            status = "청킹 실패"
            failed_ingestion.append({"code": current_stock_code, "reason": "Chunking Failed"})
            continue
        logger.info(f"텍스트 청킹 완료: {len(text_chunks)}개 청크")

        # 5. 임베딩 생성
        embeddings_success_count = 0
        for i, chunk in enumerate(text_chunks):
            if not chunk or not chunk.strip(): continue
            try:
                # get_embedding 함수가 client 인자를 받도록 수정했다면 client=openai_client 전달
                embedding = vector_store_manager.get_embedding(chunk) # 현재는 내부에서 init
                if embedding:
                    embeddings.append(embedding)
                    valid_chunks.append(chunk)
                    embeddings_success_count += 1
                else:
                    logger.warning(f"청크 {i} 임베딩 실패/빈 결과.")
                time.sleep(API_CALL_DELAY_SECONDS / 10)
            except Exception as chunk_embed_e:
                logger.error(f"청크 {i} 임베딩 중 오류: {chunk_embed_e}")
        logger.info(f"임베딩 생성 완료 ({embeddings_success_count} / {len(text_chunks)} 성공)")

        if embeddings_success_count == 0:
             logger.warning("성공한 임베딩이 없습니다. 업로드를 건너뜁니다.")
             status = "임베딩 실패"
             failed_ingestion.append({"code": current_stock_code, "reason": "Embedding Failed"})
             continue


        # 6. AI Search 업로드
        if valid_chunks and embeddings:
            # upsert 함수가 client 인자를 받도록 수정했다면 client=search_client 전달
            upload_success = vector_store_manager.upsert_documents_to_ai_search(
                text_chunks=valid_chunks,
                embeddings=embeddings,
                source_document_id=pdf_file_path # 출처 정보 전달
            )
            if not upload_success:
                logger.error(f"AI Search 업로드 실패.")
                status = "업로드 실패"
                failed_ingestion.append({"code": current_stock_code, "reason": "AI Search Upload Failed"})
            else:
                 logger.info(f"AI Search 업로드 완료.")
                 successful_ingestion_count += 1
        else:
            logger.warning(f"업로드할 유효 데이터 없음.")
            status = "업로드 데이터 없음"
            failed_ingestion.append({"code": current_stock_code, "reason": "No Valid Data to Upload"})

    except Exception as e:
        status = "예외 발생"
        logger.error(f"--- 처리 중 예외 발생 ({current_stock_code}): {e} ---")
        traceback.print_exc()
        failed_ingestion.append({"code": current_stock_code, "reason": f"Unhandled Exception: {e}"})

    finally:
        logger.info(f"--- 인덱싱 완료: {current_stock_code} (상태: {status}) ---")
        # 루프 지연 시간
        # time.sleep(API_CALL_DELAY_SECONDS) # 이미 내부에서 지연 추가됨

logger.info("===== PART 1: 데이터 인덱싱 완료 =====")
logger.info(f"총 {len(stock_codes)}개 기업 처리 시도, {successful_ingestion_count}개 성공.")
if failed_ingestion:
    logger.warning(f"인덱싱 실패 기업 ({len(failed_ingestion)}개): {failed_ingestion}")
else:
    logger.info("모든 대상 기업 인덱싱 성공!")

print(f"데이터 인덱싱 완료. 성공: {successful_ingestion_count}, 실패: {len(failed_ingestion)}")

2025-04-18 01:52:33,172 - config - INFO - ===== PART 1: 데이터 인덱싱 시작 (5개 기업) =====
2025-04-18 01:52:33,174 - config - INFO - --- 인덱싱 시작 (1/5): 005930 ---
2025-04-18 01:52:33,177 - config - INFO - PDF 로드 성공: /Users/nsj/Desktop/kospi_relation_rag_pipeline/data/input_pdfs/005930.pdf (7263840 bytes)
2025-04-18 01:52:33,178 - config - INFO - DI 클라이언트 초기화 성공
2025-04-18 01:52:33,178 - config - INFO - 바이트 데이터 분석 시작 (콘텐츠 길이: 7263840)
2025-04-18 01:52:33,179 - azure.core.pipeline.policies.http_logging_policy - INFO - Request URL: 'https://documnetintelligence0417.cognitiveservices.azure.com//documentintelligence/documentModels/prebuilt-layout:analyze?api-version=REDACTED&outputContentFormat=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Length': '7263840'
    'content-type': 'application/octet-stream'
    'Accept': 'application/json'
    'x-ms-client-request-id': '5b7dfcca-1bac-11f0-b9a4-623dbcccadd6'
    'User-Agent': 'azsdk-python-ai-documentintelligence/1.0.2 Python/3.12.7 (macO

데이터 인덱싱 완료. 성공: 5, 실패: 0


In [14]:
# Jupyter Notebook: analysis_notebook.ipynb - Cell 6 (구현 포함)

logger.info("===== PART 2 준비: 관계 추출 함수 정의 =====")

# --- 필요한 추가 임포트 ---
from azure.search.documents.models import VectorizedQuery # 벡터 쿼리용
import openai # openai 라이브러리 오류 처리 등을 위해 임포트
# config, traceback, time, re, json 등은 Cell 1에서 임포트 가정

# --- 함수 정의 ---

# 1. 관련 청크 검색 함수 구현
def retrieve_relevant_chunks(
    search_client: object,          # Cell 2에서 초기화된 SearchClient
    openai_client: object,          # Cell 2에서 초기화된 OpenAI Client (쿼리 임베딩용)
    embedding_deployment: str,   # 임베딩 배포명 (config)
    company_a_code: str,         # 종목 코드 A
    company_b_code: str,         # 종목 코드 B
    company_a_name: str,         # 한글 회사명 A
    company_b_name: str,         # 한글 회사명 B
    top_k: int = 5               # 최종 가져올 청크 수 (config)
    ) -> list[str]:
    """
    두 회사(A, B)와 관련된 텍스트 청크를 Azure AI Search에서 검색 (Hybrid Search + Semantic Reranking).
    경쟁, 소유, 공급 관련 키워드 및 벡터 검색 활용.
    """
    if not search_client or not openai_client:
        logger.error("Search Client 또는 OpenAI Client가 유효하지 않습니다.")
        return []
    logger.info(f"'{company_a_name}({company_a_code})'와 '{company_b_name}({company_b_code})' 관련 청크 검색 시작 (Top {top_k})...")

    try:
        # 1. 검색 키워드 정의 (회사명 + 관계 키워드)
        # (주의: 너무 많은 키워드는 오히려 성능 저하 유발 가능성 있음, 핵심 키워드 위주로 조정 필요)
        keywords = "경쟁 경쟁사 시장 점유율 라이벌 지분 주주 계열사 관계사 투자 소유 공급 납품 매입 매출 고객 협력사 파트너 계약 원재료 제품 competitor rival market share equity shareholder subsidiary affiliate investment ownership supply supplier customer partner agreement contract raw material product"
        search_text = f'"{company_a_name}" "{company_b_name}" {keywords}' # 종목 코드는 제외 시도 (본문 등장 빈도 낮음)
        logger.debug(f"키워드 검색어: {search_text}")

        # 2. 벡터 검색용 질문 정의 및 임베딩 생성
        query_text = f"{company_a_name}와(과) {company_b_name}의 경쟁, 소유(지분), 공급망(공급, 고객) 관계에 대한 정보는 무엇인가?"
        logger.debug(f"벡터 검색 질문: {query_text}")
        query_embedding = []
        try:
            response = openai_client.embeddings.create(model=embedding_deployment, input=query_text)
            if response.data:
                query_embedding = response.data[0].embedding
                logger.info("쿼리 텍스트 임베딩 생성 성공.")
            else:
                logger.warning("쿼리 텍스트 임베딩 생성 실패 (빈 결과).")
        except Exception as embed_e:
             logger.error(f"쿼리 텍스트 임베딩 생성 오류: {embed_e}")
             # 임베딩 실패 시 벡터 쿼리 없이 진행

        vector_query = None
        if query_embedding:
            # k_nearest_neighbors: 의미 체계 검색이 top 50개를 재정렬하므로 충분히 많은 후보군 전달 (예: 50)
            # fields: 벡터 검색 대상 필드명 (인덱스 스키마와 일치해야 함)
            vector_query = VectorizedQuery(vector=query_embedding, k_nearest_neighbors=50, fields="content_vector")
            logger.info("Vector query 생성 완료.")
        else:
            logger.warning("Query embedding이 없어 Vector query를 생성하지 않음.")

        # 3. Azure AI Search 호출 (Hybrid + Semantic)
        logger.info("Azure AI Search API 호출 시작...")
        results = search_client.search(
            search_text=search_text,                        # 키워드 쿼리
            vector_queries=[vector_query] if vector_query else None, # 벡터 쿼리 (있을 경우)
            top=top_k,                                      # 최종 반환 결과 수
            query_type="semantic",                          # 의미 체계 검색 사용
            semantic_configuration_name='my-semantic-config',# 인덱스 생성 시 정의한 이름
            select=["content", "source_document"],          # 가져올 필드
            query_caption="extractive|highlight-false",     # 문서 조각 대신 전체 문맥 강조 없는 요약 요청 (선택 사항)
            query_answer="none"                             # 답변 생성 기능은 사용 안 함
        )

        # 4. 결과 처리 (content 추출 및 중복 제거)
        retrieved_chunks = []
        unique_contents = set()
        results_count = 0
        logger.info("Search 결과 처리 시작...")
        for result in results:
            results_count += 1
            # 의미 체계 검색 점수 (참고용)
            # reranker_score = result.get('@search.reranker_score', 0)
            content = result.get('content')

            # 의미 체계 검색 요약 활용 (선택 사항)
            # caption_text = ""
            # if result.get('@search.captions'):
            #     caption = result['@search.captions'][0]
            #     if caption.highlights: # 하이라이트 대신 전체 텍스트 사용
            #         caption_text = caption.text
            #     else:
            #         caption_text = caption.text
            # # 요약과 원본 청크를 함께 사용하거나 선택적으로 사용
            # context_to_add = f"[요약] {caption_text}\n[원본] {content}" if caption_text else content

            context_to_add = content # 여기서는 원본 content만 사용
            if context_to_add and context_to_add not in unique_contents:
                retrieved_chunks.append(context_to_add)
                unique_contents.add(context_to_add)

        logger.info(f"관련 청크 {len(retrieved_chunks)}개 검색 및 중복 제거 완료 (처리된 결과 수: {results_count}).")
        logger.debug("--- retrieve_relevant_chunks 함수 종료 (정상) ---")
        return retrieved_chunks

    except Exception as e:
        logger.error(f"Azure AI Search 검색 중 오류 발생: {e}")
        traceback.print_exc()
        logger.debug("--- retrieve_relevant_chunks 함수 종료 (오류) ---")
        return []

# --- LLM 호출 함수 구현 ---
# (Tenacity Retry는 복잡성 증가로 일단 제외, 필요시 추가)
def extract_relationship_with_llm(
    openai_client: object,       # Cell 2에서 초기화된 클라이언트
    prompt: str,
    deployment_name: str      # config에서 가져온 채팅 배포명
    ) -> str | None:
    """주어진 프롬프트를 사용하여 Azure OpenAI 채팅 모델을 호출하고 응답 텍스트를 반환합니다."""
    if not openai_client: logger.error("OpenAI Client가 유효하지 않습니다."); return None
    if not deployment_name: logger.error("Chat Deployment 이름이 설정되지 않았습니다."); return None

    logger.info(f"LLM 호출 시작 (배포: {deployment_name})...")
    try:
        # Chat Completions API 사용
        response = openai_client.chat.completions.create(
            model=deployment_name, # GPT-3.5-Turbo 또는 GPT-4 배포명
            messages=[
                {"role": "system", "content": "당신은 금융 보고서 분석 전문가입니다. 제공된 컨텍스트만을 사용하여 두 회사 간의 경쟁, 소유, 공급 관계 존재 여부와 근거를 JSON 형식으로 정확히 추출해야 합니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,       # 일관성 있는 JSON 출력을 위해 0.0
            max_tokens=1024,       # 출력 토큰 제한 (JSON 크기 및 근거 문장 길이 고려)
            # response_format={"type": "json_object"} # JSON 모드 요청 (gpt-35-turbo-1106 이상, gpt-4-turbo 에서 지원)
        )

        # 응답 내용 추출
        content = None
        if response.choices:
            message = response.choices[0].message
            if message:
                content = message.content

        # 토큰 사용량 로깅
        if response.usage:
             logger.info(f"LLM 호출 성공 (토큰: In={response.usage.prompt_tokens}, Out={response.usage.completion_tokens})")
        else:
             logger.info("LLM 호출 성공 (토큰 정보 없음)")

        return content if content else None # 내용이 없는 경우 None 반환

    except openai.RateLimitError as rle:
         logger.error(f"LLM Rate Limit 오류: {rle}. 설정된 TPM 확인 및 잠시 후 재시도 필요.")
         time.sleep(20) # 예시: 20초 대기 후 None 반환
         return None
    except openai.AuthenticationError as ae:
         logger.error(f"LLM 인증 오류: {ae}. AZURE_OPENAI_KEY 또는 ENDPOINT를 확인하세요.")
         return None
    except openai.BadRequestError as bre:
         logger.error(f"LLM 잘못된 요청 오류: {bre}. 모델이 JSON 모드를 지원하는지, API 버전이 호환되는지 확인하세요.")
         return None
    except Exception as e:
        logger.error(f"LLM 호출 중 예외 발생: {e}")
        traceback.print_exc()
        return None

# --- JSON 파싱 함수 구현 ---
def parse_llm_json_output(llm_response_text: str) -> dict | None:
    """LLM 응답(JSON 문자열)을 파싱하고 기본적인 구조를 검증합니다."""
    if not llm_response_text:
        logger.warning("파싱할 LLM 응답 내용이 없습니다.")
        return None

    logger.info("LLM 응답 파싱 시도...")
    json_str = llm_response_text
    try:
        # ```json ... ``` 블록 제거 (더 견고하게)
        match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", llm_response_text, re.DOTALL | re.IGNORECASE)
        if match:
            json_str = match.group(1)
            logger.info("JSON 코드 블록에서 내용 추출.")
        else:
            # 만약 JSON 객체가 여러 줄에 걸쳐있고 코드 블록 없이 나올 경우, 첫 { 와 마지막 } 기준으로 추출 시도 (주의 필요)
            first_brace = json_str.find('{')
            last_brace = json_str.rfind('}')
            if first_brace != -1 and last_brace != -1 and last_brace > first_brace:
                 json_str = json_str[first_brace:last_brace+1]
                 logger.info("텍스트에서 첫 { 와 마지막 } 기준으로 JSON 추출 시도.")
            else:
                 logger.info("JSON 코드 블록 없어 원본 텍스트 사용 시도 (파싱 실패 가능성 있음).")

        # JSON 파싱
        relationships_data = json.loads(json_str)

        # 기본 구조 검증 강화
        if isinstance(relationships_data, dict) and \
           "company_a" in relationships_data and isinstance(relationships_data["company_a"], str) and \
           "company_b" in relationships_data and isinstance(relationships_data["company_b"], str) and \
           "relationships" in relationships_data and isinstance(relationships_data["relationships"], list):
            valid_structure = True
            # relationships 리스트 내부 구조 검증
            for item in relationships_data["relationships"]:
                if not (isinstance(item, dict) and
                        "type" in item and isinstance(item["type"], str) and
                        "connected" in item and isinstance(item["connected"], bool) and
                        "evidence" in item and isinstance(item["evidence"], list)):
                    valid_structure = False
                    logger.warning(f"relationships 리스트 내 항목 구조 이상: {item}")
                    break
            if valid_structure:
                logger.info("LLM 응답 파싱 및 구조 검증 성공")
                return relationships_data
            else:
                logger.error(f"LLM 응답 JSON 내 relationships 리스트 구조가 예상과 다릅니다.")
                return None
        else:
            logger.error(f"LLM 응답 JSON 구조가 예상과 다릅니다 (필수 키 부재 등): {relationships_data}")
            return None

    except json.JSONDecodeError as e:
        logger.error(f"LLM 응답 JSON 파싱 오류: {e}\n응답 내용 앞부분: {llm_response_text[:500]}...")
        return None
    except Exception as e:
         logger.error(f"LLM 응답 처리 중 예외 발생: {e}")
         traceback.print_exc()
         return None

print("관계 추출 관련 함수 정의 완료 (구현 포함).")

2025-04-18 14:25:44,759 - config - INFO - ===== PART 2 준비: 관계 추출 함수 정의 =====


관계 추출 관련 함수 정의 완료 (구현 포함).


In [15]:
# Jupyter Notebook: analysis_notebook.ipynb - Cell 7 (수정됨)

logger.info("===== PART 2: 관계 추출 루프 시작 =====")

# 관계 추출 대상 기업 목록 (Cell 4에서 로드 및 처리 대상 수 제한된 stock_codes 사용)
relation_process_codes = stock_codes
company_pairs = list(itertools.combinations(relation_process_codes, 2))
total_pairs = len(company_pairs)
logger.info(f"총 {total_pairs}개의 기업 쌍에 대해 관계 추출 시작...")

all_relationships = [] # 모든 추출된 관계 저장 리스트
failed_extraction_pairs = [] # 처리 실패한 쌍

# --- !!! 테스트 시에는 아래 반복 횟수를 반드시 제한하세요 !!! ---
# Cell 1의 IS_TEST_RUN과 NUM_COMPANIES_LIMIT에 따라 test_limit 자동 계산됨
test_limit = total_pairs # 전체 쌍 처리 (IS_TEST_RUN=False일 때), 테스트 시 자동으로 줄어듦 (NUM_COMPANIES_LIMIT 기반)
if IS_TEST_RUN and total_pairs > 0:
    test_limit = min(total_pairs, 10) # 테스트 시 최대 10쌍만 (또는 원하는 개수)
    logger.warning(f"테스트 실행 모드: 최대 {test_limit}개의 기업 쌍만 처리합니다.")
# -------------------------------------------------------

if total_pairs == 0:
    logger.warning("관계 추출 대상 기업 쌍이 없습니다.")
else:
    # --- 기업 쌍 반복 루프 시작 ---
    for i, (comp_a_code, comp_b_code) in enumerate(company_pairs[:test_limit]): # test_limit 적용
        logger.info(f"--- 관계 추출 진행: ({comp_a_code} vs {comp_b_code}) ({i+1}/{test_limit}) ---")
        extracted_relation_data = None # 루프마다 초기화

        # 회사 이름 조회 (Cell 4에서 생성한 code_to_name 사용)
        comp_a_name = code_to_name.get(comp_a_code, comp_a_code) # 이름 없으면 코드로 대체
        comp_b_name = code_to_name.get(comp_b_code, comp_b_code) # 이름 없으면 코드로 대체

        try:
            # --- 1. 관련 청크 검색 ---
            # Cell 6에서 정의된 retrieve_relevant_chunks 함수 호출
            # Cell 2에서 초기화된 클라이언트 및 config 값 전달
            context_chunks = retrieve_relevant_chunks(
                search_client=search_client,
                openai_client=openai_client,
                embedding_deployment=AZURE_OPENAI_EMBEDDING_DEPLOYMENT,
                company_a_code=comp_a_code,
                company_b_code=comp_b_code,
                company_a_name=comp_a_name,
                company_b_name=comp_b_name,
                top_k=TOP_K_RESULTS
            )
            if not context_chunks:
                 logger.warning("관련 청크를 찾지 못했습니다. 다음 쌍으로 건너뜁니다.")
                 failed_extraction_pairs.append(((comp_a_code, comp_b_code), "No Chunks Found"))
                 time.sleep(API_CALL_DELAY_SECONDS / 2) # 짧은 딜레이
                 continue # 루프의 다음 반복으로

            # --- 2. LLM 프롬프트 생성 ---
            # llm_extractor 모듈의 함수 사용 (Cell 1에서 import 가정)
            prompt = llm_extractor.create_relationship_extraction_prompt(
                context_chunks=context_chunks,
                company_a_name=comp_a_name, # 조회된 한글 이름 전달
                company_b_name=comp_b_name
            )
            # logger.debug(f"생성된 프롬프트 (일부): {prompt[:500]}...") # 필요시 프롬프트 확인

            # --- 3. LLM 호출 ---
            # Cell 6에서 정의된 extract_relationship_with_llm 함수 호출
            llm_response_text = extract_relationship_with_llm(
                openai_client=openai_client, # Cell 2에서 초기화
                prompt=prompt,
                deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT # config 값 (채팅 모델 배포명)
            )
            if not llm_response_text:
                logger.warning("LLM 호출에 실패했습니다. 다음 쌍으로 건너뜁니다.")
                failed_extraction_pairs.append(((comp_a_code, comp_b_code), "LLM Call Failed"))
                time.sleep(API_CALL_DELAY_SECONDS) # 실패 시 딜레이
                continue

            # --- 4. LLM 응답 파싱 ---
            # Cell 6에서 정의된 parse_llm_json_output 함수 호출
            extracted_relation_data = parse_llm_json_output(llm_response_text)
            if not extracted_relation_data:
                logger.warning("LLM 응답 파싱에 실패했습니다. 다음 쌍으로 건너뜁니다.")
                failed_extraction_pairs.append(((comp_a_code, comp_b_code), "LLM Parsing Failed"))
                continue

            # --- 5. 결과 저장 (종목 코드 추가) ---
            # 파싱된 딕셔너리에 원본 종목 코드를 추가하여 저장
            extracted_relation_data['code_a'] = comp_a_code
            extracted_relation_data['code_b'] = comp_b_code
            all_relationships.append(extracted_relation_data)
            logger.info(f"관계 추출 및 저장 성공: {comp_a_name}({comp_a_code}) vs {comp_b_name}({comp_b_code})")

        except KeyboardInterrupt:
             logger.warning("사용자에 의해 실행 중단됨.")
             raise # 중단 시 루프를 완전히 빠져나감
        except Exception as e:
            logger.error(f"--- 관계 추출 중 예외 발생 ({comp_a_code} vs {comp_b_code}): {e} ---")
            traceback.print_exc()
            failed_extraction_pairs.append(((comp_a_code, comp_b_code), f"Unhandled Exception: {e}"))

        finally:
            # 각 쌍 처리 후 지연 시간 (Rate Limit 등 고려)
            logger.debug(f"쌍 처리 후 대기: {API_CALL_DELAY_SECONDS * 2} 초")
            time.sleep(API_CALL_DELAY_SECONDS * 2) # 필요시 조절

# --- 루프 종료 ---

logger.info("===== PART 2: 관계 추출 루프 완료 =====")
logger.info(f"총 {len(all_relationships)}개의 관계 정보 추출 완료.")
if failed_extraction_pairs:
    logger.warning(f"관계 추출 실패 쌍 ({len(failed_extraction_pairs)}개): {failed_extraction_pairs[:10]}...") # 처음 10개만 로깅

print(f"관계 추출 완료. 성공 개수: {len(all_relationships)}")
# 결과 확인용 출력 (필요시 주석 해제)
# if all_relationships:
#     print("\n추출된 관계 샘플:")
#     import pprint
#     pprint.pprint(all_relationships[0])

2025-04-18 14:26:00,762 - config - INFO - ===== PART 2: 관계 추출 루프 시작 =====
2025-04-18 14:26:00,764 - config - INFO - 총 10개의 기업 쌍에 대해 관계 추출 시작...
2025-04-18 14:26:00,765 - config - WARNING - 테스트 실행 모드: 최대 10개의 기업 쌍만 처리합니다.
2025-04-18 14:26:00,768 - config - INFO - --- 관계 추출 진행: (005930 vs 000660) (1/10) ---
2025-04-18 14:26:00,768 - config - INFO - '삼성전자(005930)'와 'SK하이닉스(000660)' 관련 청크 검색 시작 (Top 5)...
2025-04-18 14:26:04,731 - httpx - INFO - HTTP Request: POST https://gptapi0417.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-04-18 14:26:04,736 - config - INFO - 쿼리 텍스트 임베딩 생성 성공.
2025-04-18 14:26:04,738 - config - INFO - Vector query 생성 완료.
2025-04-18 14:26:04,740 - config - INFO - Azure AI Search API 호출 시작...
2025-04-18 14:26:04,744 - config - INFO - Search 결과 처리 시작...
2025-04-18 14:26:04,765 - azure.core.pipeline.policies.http_logging_policy - INFO - Request URL: 'https://aiserach0417.search.windows.net/index

관계 추출 완료. 성공 개수: 0


In [13]:
#Cell 8: [PART 3] Matrix 생성 - 함수 정의 (⭐️⭐️⭐️ 구현 필요 ⭐️⭐️⭐️)
# Jupyter Notebook: full_pipeline.ipynb - Cell 8

logger.info("===== PART 3 준비: Matrix 생성 함수 정의 (구현 필요) =====")

# --- 아래 함수는 실제 로직 구현이 필요합니다 ---

def build_adjacency_matrix_from_relations(
    all_relationships: list[dict],
    company_to_idx: dict,
    num_companies: int,
    relation_types: list = ["Competition", "Supply"], # 처리할 관계 유형 지정
    strength_mapping: dict = STRENGTH_MAPPING, # config에서 가져온 매핑 사용
    use_strength: bool = False, # True: 강도 사용, False: 연결 여부(1/0) 사용
    default_value: float = 0.0
    ) -> dict: # 여러 종류의 관계 행렬 반환
    """
    추출된 모든 관계 정보와 매핑을 기반으로 인접 행렬들을 생성합니다.
    """
    if not company_to_idx:
         logger.error("회사-인덱스 매핑이 비어있어 행렬을 생성할 수 없습니다.")
         return {}
    logger.info(f"추출된 관계 기반 {relation_types} 인접 행렬 생성 시작 (연결강도 사용: {use_strength})...")
    # === 실제 구현 필요 ===
    # 1. 관계 유형별 희소 행렬(LIL 추천) 초기화
    matrices = {rtype: sp.lil_matrix((num_companies, num_companies), dtype=np.float32) for rtype in relation_types}
    processed_relations = 0

    # 2. all_relationships 순회
    for rel_data in all_relationships:
        try:
            comp_a_name = rel_data.get("company_a") # LLM 결과 내 이름
            comp_b_name = rel_data.get("company_b") # LLM 결과 내 이름
            relationships = rel_data.get("relationships", [])
            if not comp_a_name or not comp_b_name or not relationships: continue

            # 이름을 인덱스로 변환 (매핑에 이름이 없으면 건너뜀)
            # 주의: LLM이 반환한 이름과 stock_code가 다를 수 있으므로 매핑 방식 주의!
            # 여기서는 stock_code가 이름이라고 가정
            if comp_a_name not in company_to_idx or comp_b_name not in company_to_idx:
                continue
            idx_a = company_to_idx[comp_a_name]
            idx_b = company_to_idx[comp_b_name]

            # 3. 각 관계 정보 처리
            for relationship in relationships:
                rel_type = relationship.get("type")
                if rel_type in matrices: # 정의된 관계 유형만 처리
                    weight = default_value
                    if use_strength: # 강도 사용 시
                        strength_str = relationship.get("strength", "None")
                        weight = float(strength_mapping.get(strength_str, default_value))
                    else: # 연결 여부 사용 시
                        connected = relationship.get("connected", False)
                        weight = 1.0 if connected else default_value

                    # 4. 가중치가 있으면 행렬 값 설정
                    if weight > default_value:
                        matrices[rel_type][idx_a, idx_b] = weight
                        processed_relations += 1
                        # 필요시 matrix[idx_b, idx_a] = weight 설정 (대칭 관계)

        except Exception as e:
            logger.error(f"관계 데이터 처리 중 오류: {rel_data}, 오류: {e}")

    # 5. 결과 행렬 반환 (COO 포맷 변환)
    final_matrices = {rtype: mat.tocoo() for rtype, mat in matrices.items()}
    logger.info(f"인접 행렬 생성 완료 ({processed_relations}개 관계 반영). 행렬 종류: {list(final_matrices.keys())}")
    return final_matrices
    # ====================

print("Matrix 생성 관련 함수 정의 완료 (내부 구현 필요)")

2025-04-18 14:25:33,764 - config - INFO - ===== PART 3 준비: Matrix 생성 함수 정의 (구현 필요) =====


Matrix 생성 관련 함수 정의 완료 (내부 구현 필요)


In [ ]:
#Cell 9: [PART 3] Matrix 생성 및 저장
# Jupyter Notebook: full_pipeline.ipynb - Cell 9

logger.info("===== PART 3: 인접 행렬 생성 및 저장 시작 =====")
adjacency_matrices = {} # 초기화

# 이전 단계(Cell 7)에서 생성된 all_relationships 사용
if 'all_relationships' in locals() and all_relationships and num_companies > 0:
    try:
        # Matrix 생성 함수 호출 (구현된 함수 사용)
        adjacency_matrices = build_adjacency_matrix_from_relations(
            all_relationships=all_relationships,
            company_to_idx=company_to_idx, # Cell 4에서 생성됨
            num_companies=num_companies,  # Cell 4에서 정의됨
            # relation_types=["Competition", "Supply"], # 처리할 타입 지정 가능
            use_strength=False # True로 바꾸면 강도값 사용, False면 연결 여부(1/0) 사용
        )

        # 각 관계 유형별로 행렬 저장
        saved_count = 0
        for relation_type, matrix in adjacency_matrices.items():
            if matrix.nnz > 0: # 내용이 있는 행렬만 저장
                # 파일 이름에 행렬 크기 포함
                matrix_file_name = f"{relation_type}_adjacency_{num_companies}x{num_companies}.npz"
                matrix_file_path = os.path.join(OUTPUT_MATRIX_DIR, matrix_file_name)
                # matrix_builder.save_matrix 함수 사용 (내부에서 sp.save_npz 호출 가정)
                matrix_builder.save_matrix(matrix, matrix_file_path) # COO 포맷 전달
                logger.info(f"'{relation_type}' 인접 행렬 저장 완료 (파일: {matrix_file_path}, non-zero: {matrix.nnz})")
                saved_count += 1
            else:
                logger.warning(f"'{relation_type}' 관계 정보가 없어 행렬을 저장하지 않습니다.")

        if saved_count > 0:
             logger.info(f"--- 총 {saved_count}개의 인접 행렬 저장 완료 ---")
        else:
             logger.info("--- 저장할 유효한 인접 행렬이 없습니다 ---")

    except Exception as e:
        logger.error(f"--- Matrix 생성 또는 저장 중 오류 발생: {e} ---")
        traceback.print_exc()
else:
    logger.warning("--- Matrix를 생성할 관계 데이터가 없거나 처리 대상 기업이 없습니다. ---")

logger.info("===== 전체 파이프라인 노트북 실행 완료 =====")
print(f"Matrix 생성 및 저장 완료. 생성된 행렬 종류: {list(adjacency_matrices.keys())}")